In [264]:
# Pandas will be the main library used to clean and present the data 
# before moving it over to Tableau for the
# final visualizations

# Pandas library and `read_csv` method have both been imported
# as I would like to have cleaner code and can call the method
# without the extra syntax
import pandas as pd 
from pandas import read_csv 
from tabulate import *
# Assigning variable to the datasets
# These names will represent the three different 
# data sets used to create this project
single_family_data = read_csv('./datasets/single_family_home.csv')
condo_data = read_csv('./datasets/median_condo_price.csv')
black_and_latino_data = read_csv('./datasets/black_and_latino_mortgage_rates.csv')
eviction_data = read_csv('./datasets/evictions.csv')
single_family_top_performers = read_csv('./datasets/single_fam_top_performers.csv')

# Format floats to show 2 decimal places for '% Change' col
pd.options.display.float_format = '{:.2f}'.format

# Prints a message to show that everything is loaded up
# print("All systems GO!")

In [265]:
# convert inputs in the '% change' column to float type
# this allows us to properly sort it by numerical value
# whereas before, they were being treated as string values

# Removing of the % symbol needs to happen first
single_family_data['% Change'] = single_family_data['% Change'].str[:-1]
condo_data['Condo.PercChange'] = condo_data['Condo.PercChange'].str[:-1]
black_and_latino_data['Percent of Home Loans to Black and Latino Buyers'] = black_and_latino_data['Percent of Home Loans to Black and Latino Buyers'].str[:-1]

# Converts the type to a float from `object`
single_family_data['% Change'] = single_family_data['% Change'].astype(float)
condo_data['Condo.PercChange'] = condo_data['Condo.PercChange'].astype(float)
black_and_latino_data['Percent of Home Loans to Black and Latino Buyers'] = black_and_latino_data['Percent of Home Loans to Black and Latino Buyers'].astype(float)


# Format floats to show 2 decimal places for '% Change' col
pd.options.display.float_format = '{:.2f}'.format

# Deletes notes column as its empty aside for one row

# The notes column only had NaN inputs, except
# for one row, which was an anomaly
del single_family_data['Notes']

## Single Family Median Price Data:

In this section we will be combing through the single family median price dataset for the months of Jan-Jun of 2021 and the same months of 2022.

We have no aim as to what we hope to uncover, only merely to present the data in such a way that it makes full sense to the end user. 

In [266]:
# The describe method allows us to get 
# some insight into the overall dataset and 
# also gives us some key pieces of information 

single_family_data.describe()

,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
count,147.00,147.00,147.00
mean,721655.58,805912.24,10.79
std,380077.98,474227.22,9.58
min,347500.00,370250.00,-21.20
25%,491250.00,547000.00,5.65
50%,615000.00,660000.00,11.20
75%,801250.00,865000.00,15.60
max,3462500.00,4400000.00,44.40


In [267]:
# Top 5 performing municipalities overall
# these are not filtered by anything other than 
# 
# single_family_data.sort_values(by='% Change', ascending=False).head()

# Upon my research I have found a new way of sorting 
# by calling the nLargest method

single_family_data.nlargest(5, '% Change')

# Prints out markdown table to allow for smooth transition to readme
print(single_family_data.nsmallest(5, '% Change').to_markdown(tablefmt="grid"))

+-----+----------------+--------------------+--------------------------------------+--------------------------------------+------------+
|     | Municipality   | Communities Type   |   Median Sale Price, Jan - June 2021 |   Median Sale Price, Jan - June 2022 |   % Change |
+=====+================+====================+======================================+======================================+============+
| 124 | Rockport       | Developing Suburbs |                               837500 |                               660000 |      -21.2 |
+-----+----------------+--------------------+--------------------------------------+--------------------------------------+------------+
| 126 | Groveland      | Maturing Suburbs   |                               615000 |                               521500 |      -15.2 |
+-----+----------------+--------------------+--------------------------------------+--------------------------------------+------------+
|  92 | Ipswich        | Developing Subur

In [268]:
# Worst 5 municipalities overall
# Printed in markdown to allow for smooth transfer to readme

print(single_family_data.nsmallest(5, '% Change').to_markdown(tablefmt="grid"))

+-----+----------------+--------------------+--------------------------------------+--------------------------------------+------------+
|     | Municipality   | Communities Type   |   Median Sale Price, Jan - June 2021 |   Median Sale Price, Jan - June 2022 |   % Change |
+=====+================+====================+======================================+======================================+============+
| 124 | Rockport       | Developing Suburbs |                               837500 |                               660000 |      -21.2 |
+-----+----------------+--------------------+--------------------------------------+--------------------------------------+------------+
| 126 | Groveland      | Maturing Suburbs   |                               615000 |                               521500 |      -15.2 |
+-----+----------------+--------------------+--------------------------------------+--------------------------------------+------------+
|  92 | Ipswich        | Developing Subur

In [269]:
# Presenting data by community type
metro_data_single = single_family_data.loc[single_family_data['Communities Type'] == "Metro Core Communities"]
reg_urban_centers_single = single_family_data.loc[single_family_data['Communities Type'] == "Regional Urban Centers"]
street_car_suburb_single = single_family_data.loc[single_family_data['Communities Type'] == "Streetcar Suburbs"]
developing_suburb_single = single_family_data.loc[single_family_data['Communities Type'] == "Developing Suburbs"]
maturing_suburb_single = single_family_data.loc[single_family_data['Communities Type'] == "'Maturing Suburbs"]
rural_town_single = single_family_data.loc[single_family_data['Communities Type'] == "Rural Towns"]

In [270]:
# Sorting by community type column
# to present which type experienced
# the greatest increase in value by % 
single_metro_sorted_by_pct_change =  metro_data_single.nlargest(5, '% Change')
single_reg_urban_pct_change = reg_urban_centers_single.nlargest(5, '% Change')
single_street_car_suburn_pct_change = street_car_suburb_single.nlargest(5, '% Change')
single_dev_suburb_pct_change = developing_suburb_single.nlargest(5, '% Change')
maturing_suburb_single.nlargest(5, '% Change')
rural_town_single.nlargest(1, '% Change')

,Municipality,Communities Type,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
145,Ashby,Rural Towns,350000,370250,5.80


### Overview of Single Family home data by community type

In [271]:
# Metro Core Community
metro_data_single.describe()

,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
count,7.00,7.00,7.00
mean,1135660.71,1399028.57,20.76
std,1091096.60,1397197.62,12.93
min,476500.00,577450.00,8.60
25%,531000.00,605375.00,11.25
50%,580000.00,630000.00,15.40
75%,1184312.50,1487500.00,27.20
max,3462500.00,4400000.00,44.40


In [272]:
# metro_data_single.sort_values(by='% Change', ascending=False)
metro_data_single.nlargest(7, '% Change')

,Municipality,Communities Type,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
8,Somerville,Metro Core Communities,831125,1200000,44.40
27,Chelsea,Metro Core Communities,476500,606500,27.30
0,Boston,Metro Core Communities,3462500,4400000,27.10
1,Cambridge,Metro Core Communities,1537500,1775000,15.40
20,Everett,Metro Core Communities,537000,604250,12.50
16,Revere,Metro Core Communities,525000,577450,10.00
11,Malden,Metro Core Communities,580000,630000,8.60


In [273]:
# Regional Urban Centers Single Family
reg_urban_centers_single.describe()

,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
count,17.00,17.00,17.00
mean,520102.94,567195.24,9.46
std,102287.47,103337.72,4.59
min,360000.00,413000.00,-0.80
25%,445000.00,500000.00,5.80
50%,529000.00,590000.00,10.30
75%,600000.00,620000.00,13.30
max,750000.00,815000.00,15.70


In [274]:
reg_urban_centers_single.nlargest(5, '% Change')

,Municipality,Communities Type,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
78,Amesbury,Regional Urban Centers,481000,556319,15.70
6,Lawrence,Regional Urban Centers,360000,413000,14.70
3,Brockton,Regional Urban Centers,377500,430000,13.90
22,Salem,Regional Urban Centers,529000,600000,13.40
5,Lynn,Regional Urban Centers,450000,510000,13.30


In [275]:
street_car_suburb_single.describe()

,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
count,9.00,9.00,9.00
mean,1039227.78,1198755.56,12.99
std,439110.74,611463.45,10.78
min,629000.00,694900.00,1.60
25%,728050.00,802500.00,7.40
50%,790000.00,850000.00,9.70
75%,1441500.00,1575900.00,14.30
max,1850000.00,2542000.00,37.40


In [276]:
street_car_suburb_single.nlargest(5, '% Change')

,Municipality,Communities Type,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
13,Brookline,Streetcar Suburbs,1850000,2542000,37.40
21,Arlington,Streetcar Suburbs,949500,1156000,21.70
15,Medford,Streetcar Suburbs,728050,832500,14.30
72,Winthrop,Streetcar Suburbs,629000,694900,10.50
43,Melrose,Streetcar Suburbs,775000,850000,9.70


In [277]:
developing_suburb_single.describe()

,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
count,65.00,65.00,65.00
mean,636039.23,687538.46,8.87
std,225127.99,242892.58,10.23
min,347500.00,400000.00,-21.20
25%,461000.00,517500.00,3.60
50%,583450.00,622500.00,9.50
75%,747500.00,764500.00,14.50
max,1425000.00,1631500.00,33.50


In [278]:
developing_suburb_single.nlargest(5, '% Change')

,Municipality,Communities Type,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
141,Sherborn,Developing Suburbs,955000,1275000,33.50
79,Abington,Developing Suburbs,433000,550000,27.00
93,Medway,Developing Suburbs,532500,665000,24.90
116,Ayer,Developing Suburbs,418000,521500,24.80
123,Stow,Developing Suburbs,669000,833000,24.50


In [279]:
rural_town_single.describe()

,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
count,1.00,1.00,1.00
mean,350000.00,370250.00,5.80
std,NaN,NaN,NaN
min,350000.00,370250.00,5.80
25%,350000.00,370250.00,5.80
50%,350000.00,370250.00,5.80
75%,350000.00,370250.00,5.80
max,350000.00,370250.00,5.80


In [280]:
del single_family_top_performers['Notes']
single_family_top_performers['% Change'] = single_family_top_performers['% Change'].str[:-1]
single_family_data['% Change'] = single_family_data['% Change'].astype(float)
single_family_top_performers


,Municipality,Communities Type,"Median Sale Price, Jan - June 2021","Median Sale Price, Jan - June 2022",% Change
0,Somerville,Metro Core Communities,831125,1200000,44.4
1,Brookline,Streetcar Suburbs,1850000,2542000,37.4
2,Amesbury,Regional Urban Centers,481000,556319,15.7
3,Sherborn,Developing Suburbs,955000,1275000,33.5
4,Ashby,Rural Towns,350000,370250,05.8


## Condo Sales Data


In [281]:
# Getting a brief overview of the dataset
condo_data.describe()

#As seen below, running the above command
# We can see that we are working with the same qty of data inputs
# as for the single family homes with different col names
# we have already formatted the 'Condo.PercChange' column to allow for dtype
# changing which will allow us to more easily and correctly filter the data
# #

condo_data.rename(columns = {'Condo.PercChange':'% Change'}, inplace = True)
condo_data.columns
condo_data.head()

,Town,Community Type,Condo.2021,Condo.2022,% Change
0,Abington,Developing Suburbs,349000,400000,14.60
1,Acton,Maturing Suburbs,370000,376000,1.60
2,Amesbury,Regional Urban Centers,340000,360500,6.00
3,Andover,Developing Suburbs,374995,455000,21.30
4,Arlington,Streetcar Suburbs,695000,812500,16.90


In [282]:
# Presenting data by community type
metro_data_condo = condo_data.loc[lambda condo_data: condo_data['Community Type'] == "Metro Core Communities"]
reg_urban_centers_condo = condo_data.loc[lambda condo_data: condo_data['Community Type'] == "Regional Urban Centers"]
street_car_suburb_condo = condo_data.loc[lambda condo_data: condo_data['Community Type'] == "Streetcar Suburbs"]
developing_suburb_condo= condo_data.loc[lambda condo_data: condo_data['Community Type'] == "Developing Suburbs"]
maturing_suburb_condo = condo_data.loc[lambda condo_data: condo_data['Community Type'] == "'Maturing Suburbs"]
rural_town_condo = condo_data.loc[lambda condo_data: condo_data['Community Type'] == "Rural Towns"]

In [283]:
metro_data_condo.describe()

,Condo.2021,Condo.2022,% Change
count,7.00,7.00,7.00
mean,588778.57,634342.86,6.04
std,215419.48,265963.73,6.56
min,385500.00,407000.00,-3.20
25%,423500.00,423000.00,2.15
50%,440000.00,445000.00,5.60
75%,788475.00,871250.00,10.45
max,872000.00,999900.00,14.70


In [284]:
metro_data_condo.nlargest(5, '% Change')

,Town,Community Type,Condo.2021,Condo.2022,% Change
14,Boston,Metro Core Communities,872000,999900,14.70
22,Cambridge,Metro Core Communities,802000,900000,12.20
117,Somerville,Metro Core Communities,774950,842500,8.70
67,Malden,Metro Core Communities,385500,407000,5.60
105,Revere,Metro Core Communities,422000,445000,5.50


In [285]:
reg_urban_centers_condo.describe()

,Condo.2021,Condo.2022,% Change
count,17.00,17.00,17.00
mean,345802.94,371252.94,8.60
std,97849.97,97962.14,10.68
min,182750.00,220000.00,-10.40
25%,285000.00,280000.00,3.70
50%,340000.00,360500.00,7.50
75%,425000.00,430000.00,15.50
max,499950.00,577000.00,31.30


In [286]:
reg_urban_centers_condo.nlargest(5, '% Change')

,Town,Community Type,Condo.2021,Condo.2022,% Change
60,Lawrence,Regional Urban Centers,182750,240000,31.30
80,Methuen,Regional Urban Centers,288000,344000,19.40
50,Haverhill,Regional Urban Centers,285000,338000,18.60
65,Lynn,Regional Urban Centers,288000,338500,17.50
12,Beverly,Regional Urban Centers,355000,410000,15.50


In [287]:
street_car_suburb_condo.describe()

,Condo.2021,Condo.2022,% Change
count,9.00,9.00,9.00
mean,636827.78,686905.56,7.50
std,144175.10,168548.47,6.61
min,465000.00,450000.00,-3.50
25%,525000.00,550000.00,3.20
50%,589950.00,661250.00,6.20
75%,780000.00,812500.00,12.10
max,850000.00,905000.00,16.90


In [288]:
street_car_suburb_condo.nlargest(5, '% Change')

,Town,Community Type,Condo.2021,Condo.2022,% Change
4,Arlington,Streetcar Suburbs,695000,812500,16.90
90,Newton,Streetcar Suburbs,780000,905000,16.00
131,Watertown,Streetcar Suburbs,589950,661250,12.10
144,Winthrop,Streetcar Suburbs,465000,505000,8.60
76,Medford,Streetcar Suburbs,580000,615900,6.20


In [289]:
developing_suburb_condo.describe()

,Condo.2021,Condo.2022,% Change
count,65.00,65.00,65.00
mean,382301.72,449622.62,9.46
std,214213.72,244286.00,22.34
min,0.00,0.00,-31.80
25%,295000.00,349250.00,-3.00
50%,367500.00,455000.00,1.20
75%,480000.00,570000.00,15.60
max,1000000.00,985000.00,120.60


In [290]:
developing_suburb_condo.nlargest(5, '% Change')

,Town,Community Type,Condo.2021,Condo.2022,% Change
53,Holliston,Developing Suburbs,232500,513000,120.60
146,Wrentham,Developing Suburbs,413000,748372,81.20
82,Middleton,Developing Suburbs,485000,844698,74.20
28,Cohasset,Developing Suburbs,700813,985000,40.60
137,Westford,Developing Suburbs,415000,570000,37.30


In [291]:
rural_town_condo

,Town,Community Type,Condo.2021,Condo.2022,% Change
5,Ashby,Rural Towns,0,0,-3.00


In [292]:
condo_top_performers = condo_data.loc[condo_data['Community Type'] == "Metro Core Communities"]
condo_top_performers.nlargest(5, '% Change')

,Town,Community Type,Condo.2021,Condo.2022,% Change
14,Boston,Metro Core Communities,872000,999900,14.70
22,Cambridge,Metro Core Communities,802000,900000,12.20
117,Somerville,Metro Core Communities,774950,842500,8.70
67,Malden,Metro Core Communities,385500,407000,5.60
105,Revere,Metro Core Communities,422000,445000,5.50


# Black and Latino Owner Share


In [293]:
black_and_latino_data.sort_values(by='Percent of Home Loans to Black and Latino Buyers', ascending=False)


,Municipality,Community Type,Percent of Home Loans to Black and Latino Buyers
6,Lawrence,Regional Urban Centers,89.20
3,Brockton,Regional Urban Centers,65.90
16,Revere,Metro Core Communities,54.30
33,Randolph,Maturing Suburbs,53.70
5,Lynn,Regional Urban Centers,53.60
...,...,...,...
76,Concord,Maturing Suburbs,1.00
75,Newburyport,Regional Urban Centers,0.90
138,Wenham,Developing Suburbs,0.00
142,Essex,Developing Suburbs,0.00


In [294]:
# Presenting data by community type
b_l_share_metro_data = black_and_latino_data.loc[black_and_latino_data['Community Type'] == "Metro Core Communities"]
b_l_share_reg_urban_centers = black_and_latino_data.loc[black_and_latino_data['Community Type'] == "Regional Urban Centers"]
b_l_share_street_car_suburb = black_and_latino_data.loc[black_and_latino_data['Community Type'] == "Streetcar Suburbs"]
b_l_share_developing_suburb = black_and_latino_data.loc[black_and_latino_data['Community Type'] == "Developing Suburbs"]
b_l_share_maturing_suburb = black_and_latino_data.loc[black_and_latino_data['Community Type'] == "'Maturing Suburbs"]
b_l_share_rural_town = black_and_latino_data.loc[black_and_latino_data['Community Type'] == "Rural Towns"]

In [295]:
b_l_share_metro_data.describe()

,Percent of Home Loans to Black and Latino Buyers
count,7.00
mean,22.07
std,18.27
min,4.60
25%,7.90
50%,15.70
75%,32.05
max,54.30


In [296]:
b_l_share_metro_data.nlargest(5, 'Percent of Home Loans to Black and Latino Buyers')

,Municipality,Community Type,Percent of Home Loans to Black and Latino Buyers
16,Revere,Metro Core Communities,54.30
20,Everett,Metro Core Communities,32.10
27,Chelsea,Metro Core Communities,32.00
11,Malden,Metro Core Communities,15.70
0,Boston,Metro Core Communities,10.90


In [297]:
b_l_share_reg_urban_centers.describe()

,Percent of Home Loans to Black and Latino Buyers
count,17.00
mean,23.73
std,25.12
min,0.90
25%,5.70
50%,14.60
75%,27.60
max,89.20


In [298]:
b_l_share_reg_urban_centers.nlargest(5, 'Percent of Home Loans to Black and Latino Buyers')

,Municipality,Community Type,Percent of Home Loans to Black and Latino Buyers
6,Lawrence,Regional Urban Centers,89.20
3,Brockton,Regional Urban Centers,65.90
5,Lynn,Regional Urban Centers,53.60
19,Methuen,Regional Urban Centers,40.40
10,Haverhill,Regional Urban Centers,27.60


In [299]:
b_l_share_street_car_suburb.describe()

,Percent of Home Loans to Black and Latino Buyers
count,9.00
mean,4.79
std,2.85
min,2.60
25%,2.90
50%,3.80
75%,5.70
max,11.50


In [300]:
b_l_share_developing_suburb.describe()


,Percent of Home Loans to Black and Latino Buyers
count,65.00
mean,5.12
std,4.02
min,0.00
25%,2.00
50%,3.90
75%,6.60
max,17.30


In [301]:
b_l_share_rural_town.describe()

,Percent of Home Loans to Black and Latino Buyers
count,1.00
mean,12.00
std,NaN
min,12.00
25%,12.00
50%,12.00
75%,12.00
max,12.00


# Eviction dataset

In [302]:
eviction_data['Community Type'].unique()

array(['Developing Suburbs', 'Maturing Suburbs', 'Regional Urban Centers',
       'Streetcar Suburb', 'Rural Towns', 'Metro Core Community'],
      dtype=object)

In [303]:
# Presenting data by community type
eviction_metro = eviction_data.loc[ eviction_data['Community Type'] == "Metro Core Community"]
eviction_reg_urban_centers = eviction_data.loc[eviction_data['Community Type'] == "Regional Urban Centers"]
eviction_street_car_suburb = eviction_data.loc[eviction_data['Community Type'] == "Streetcar Suburb"]
eviction_developing_suburb = eviction_data.loc[eviction_data['Community Type'] == "Developing Suburbs"]
eviction_maturing_suburb = eviction_data.loc[eviction_data['Community Type'] == "'Maturing Suburbs"]
eviction_rural_town = eviction_data.loc[eviction_data['Community Type'] == "Rural Towns"]

In [304]:
eviction_metro.describe()

,Jan-June 2022,Eviction Rate
count,7.00,7.00
mean,172.71,63.17
std,241.98,36.31
min,33.00,14.60
25%,77.00,32.60
50%,80.00,81.00
75%,112.00,85.70
max,718.00,110.00


In [305]:
eviction_metro.nlargest(5, 'Eviction Rate')

,Municipality or Neighborhood,Community Type,County,Jan-June 2022,Renter Units,Eviction Rate
104,Revere,Metro Core Community,Suffolk,103,"9,361",110.00
66,Malden,Metro Core Community,Middlesex,121,"13,542",89.40
37,Everett,Metro Core Community,Middlesex,80,"9,761",82.00
26,Chelsea,Metro Core Community,Suffolk,78,"9,635",81.00
14,Boston,Metro Core Community,Suffolk,718,"176,686",40.60


In [306]:
eviction_reg_urban_centers.describe()

,Jan-June 2022,Eviction Rate
count,17.00,17.00
mean,97.76,92.59
std,72.50,42.85
min,6.00,32.50
25%,36.00,55.70
50%,87.00,82.90
75%,166.00,127.30
max,227.00,160.40


In [307]:
eviction_reg_urban_centers.nlargest(5, 'Eviction Rate')

,Municipality or Neighborhood,Community Type,County,Jan-June 2022,Renter Units,Eviction Rate
39,Framingham,Regional Urban Centers,Middlesex,200,"12,465",160.40
94,Norwood,Regional Urban Centers,Norfolk,87,"5,452",159.60
70,Marlborough,Regional Urban Centers,Middlesex,99,"6,752",146.60
19,Brockton,Regional Urban Centers,Plymouth,201,"13,837",145.30
64,Lynn,Regional Urban Centers,Essex,227,"17,837",127.30


In [308]:
eviction_street_car_suburb.describe()

,Jan-June 2022,Eviction Rate
count,9.00,9.00
mean,21.78,23.54
std,24.58,19.80
min,0.00,0.00
25%,6.00,9.30
50%,12.00,18.80
75%,29.00,33.10
max,78.00,65.30


In [309]:
eviction_street_car_suburb.nlargest(5, 'Eviction Rate')

,Municipality or Neighborhood,Community Type,County,Jan-June 2022,Renter Units,Eviction Rate
127,Waltham,Streetcar Suburb,Middlesex,78,"11,946",65.30
75,Medford,Streetcar Suburb,Middlesex,39,"10,604",36.80
89,Newton,Streetcar Suburb,Middlesex,29,"8,768",33.10
129,Watertown,Streetcar Suburb,Middlesex,20,"7,886",25.40
77,Melrose,Streetcar Suburb,Middlesex,7,"3,725",18.80


In [310]:
eviction_developing_suburb.describe()

,Jan-June 2022,Eviction Rate
count,65.00,65.00
mean,6.89,56.45
std,10.90,60.96
min,0.00,0.00
25%,0.00,0.00
50%,3.00,37.70
75%,7.00,91.10
max,53.00,256.00


In [311]:
eviction_developing_suburb.nlargest(5, 'Eviction Rate')

,Municipality or Neighborhood,Community Type,County,Jan-June 2022,Renter Units,Eviction Rate
143,Wrentham,Developing Suburbs,Norfolk,17,664,256.00
18,Bridgewater,Developing Suburbs,Plymouth,53,"2,223",238.40
24,Carver,Developing Suburbs,Plymouth,8,442,181.00
128,Wareham,Developing Suburbs,Plymouth,42,"2,539",165.40
106,Rockland,Developing Suburbs,Plymouth,28,"1,746",160.40


In [312]:
eviction_rural_town

,Municipality or Neighborhood,Community Type,County,Jan-June 2022,Renter Units,Eviction Rate
5,Ashby,Rural Towns,Middlesex,0,84,0.00


In [313]:
# eviction_developing_suburb.to_csv('developing_suburbs.csv')